### Code to Extract Test Metrics
#### Paper Used: "A Test-suite Diagnosability Metric for Spectrum-based Fault Localization Approaches" for DDU metrics

In [1]:
import csv
import itertools
from collections import Counter

In [2]:
#writing to csv
def matchList_writer(csv_file,csv_columns,dict_data):
    with open(csv_file,'w') as csvFile:
        writer=csv.DictWriter(csvFile, fieldnames=csv_columns)
        writer.writeheader()
        for data in dict_data:
            writer.writerow(data)
    return


In [3]:
#finds ranking between each methods
def findRanking(array):
    _,id = np.unique(np.asarray(array),return_inverse=True)
    result_array = (id.max() - id + 1).reshape(np.asarray(array).shape)
    return result_array



In [4]:
#diversity equation applied from the DDU paper
def find_diversity(matrixpath):
    ones=0
    zeros=0
    total = 0
    count=0
    f=open(matrixpath,"r")

    array=[]
    for line in f:
        r=line.replace(" ", "")
       # print(r.strip())

        if r.strip().endswith("1"):
            r = r.strip()[:-1] + "+"
        elif r.strip().endswith("0"):
            r = r.strip()[:-1] + "-"
        array.append(r.strip())
        
    row_length=len(array)
    column_length=len(array[0])
    equal_list = []
    total_list = []
    
    for i in range(0,column_length):    
        for j in range(0,row_length):
            x = array[j][i]
            equal_list.append(x)
       # print(equal_list)
        total_list.append(equal_list)
        equal_list = []
   # print(total_list)

    not_unique_list = []
    unique_list = []
    for el in total_list:
        if el not in unique_list:
            unique_list.append(el)
        else:
            not_unique_list.append(el)
    count = len(not_unique_list)        
   # print (count)
    diversity = 1 - (count*(count-1))/(column_length*(column_length-1))
   # print(diversity)
    return diversity



In [5]:
#uniqueness equation applied from the DDU paper
def find_uniqueness(matrixpath):
    ones=0
    zeros=0
    total = 0
    count=0
    f=open(matrixpath,"r")

    array=[]
    for line in f:
        r=line.replace(" ", "")
       # print(r.strip())

        if r.strip().endswith("1"):
            r = r.strip()[:-1] + "+"
        elif r.strip().endswith("0"):
            r = r.strip()[:-1] + "-"
        array.append(r.strip())
        
    row_length=len(array)
    column_length=len(array[0])
    equal_list = []
    total_list = []
    
    for i in range(0,column_length):    
        for j in range(0,row_length):
            x = array[j][i]
            equal_list.append(x)
       # print(equal_list)
        total_list.append(equal_list)
        equal_list = []
   # print(total_list) 
    
    not_unique_list = []
    unique_list = []
    for el in total_list:
        if el not in unique_list:
            unique_list.append(el)
        else:
            not_unique_list.append(el)


    count = len(not_unique_list)

    uniqueness = (column_length-count)/column_length
   # print(uniqueness)
    return uniqueness

In [6]:
# density equation applied from the DDU paper
def find_density(matrixpath):
    ones=0
    zeros=0
    total = 0
    f=open(matrixpath,"r")

    array=[]
    for line in f:
        r=line.replace(" ", "")
       # print(r.strip())

        if r.strip().endswith("1"):
            r = r.strip()[:-1] + "+"
        elif r.strip().endswith("0"):
            r = r.strip()[:-1] + "-"
        array.append(r.strip())
    

    row_length=len(array)
    column_length=len(array[0])
    
    for i in range(0,column_length):    
        for j in range(0,row_length):
            if array[j][i] == "1":
                ones=ones+1
            elif array[j][i] == "0":
                zeros = zeros+1
                
    total = ones + zeros
    ro = ones/(row_length*column_length)
    ro_prime = 1 - abs(1 - 2*ro)
    #print(ro_prime)
    return ro_prime

In [7]:
#Calculates the total number of passed test cases and the total number of failed test cases

def find_pass_fail(matrixpath):
    f=open(matrixpath,"r")

    array=[]
    for line in f:
        r=line.replace(" ", "")
       # print(r.strip())

        if r.strip().endswith("1"):
            r = r.strip()[:-1] + "+"
        elif r.strip().endswith("0"):
            r = r.strip()[:-1] + "-"
        array.append(r.strip())
    #print(array)

    row_length=len(array)
    column_length=len(array[0])
   # print(row_length)

    passed = 0;
    failed = 0;
 
    for j in range(0,row_length):
       # print("array[j]", j)
        isPassed=array[j][column_length-1]
       # print("after", j)
        if isPassed == '+':
            passed=passed+1
        elif isPassed=='-':
            failed=failed+1

    return passed,failed



In [8]:
# Calculate the total number of failed test cases that don’t cover the statement
# Calculate the total number of failed test cases that cover the statement
# Calculate the total number of successful test cases that cover the statement as for the dstar algorithm
def find_ncf_nuf_ncs(matrixpath):
    f=open(matrixpath,"r")

    array=[]
    for line in f:
        r=line.replace(" ", "")
       # print(r.strip())

        if r.strip().endswith("1"):
            r = r.strip()[:-1] + "+"
        elif r.strip().endswith("0"):
            r = r.strip()[:-1] + "-"
        array.append(r.strip())
    #print(array)

    row_length=len(array)
    column_length=len(array[0])


    Ncf=0
    Nuf=0
    Ncs=0
    Ncf_list = list()
    Nuf_list = list()
    Ncs_list= list()
    dstarArray=[]
 
    for i in range(0,column_length):
        
        for j in range(0,row_length):
            isPassed=array[j][column_length-1]
            if isPassed == '-' and array[j][i]=='1':
                Ncf=Ncf+1
            elif isPassed=='-' and array[j][i]=='0':
                Nuf=Nuf+1
            elif isPassed=='+' and array[j][i]=='1':
                Ncs=Ncf+1   
      
    return Ncf,Nuf,Ncs



In [9]:
def get_suspiciousness_and_rank(gzoltarpath,projectName):
    matchList=[]
    passlist = list()
    faillist = list ()
    Ncf_list = list()
    Nuf_list = list()
    Ncs_list = list()
    
                 
    for i in range(0,versionCount):
        d={}
        matrixPath=""
        ## change the matrix path accordingly***
        matrixPath=gzoltarpath+"/"+projectName + "/math" +"/"+str(i+1)+"/matrix"
        passed, failed = find_pass_fail(matrixPath)
        passlist.append(passed)
        faillist.append(failed)
        Ncf, Nuf, Ncs = find_ncf_nuf_ncs(matrixPath)
        density = find_density(matrixPath)
        uniqueness = find_uniqueness(matrixPath)
        diversity = find_diversity(matrixPath)
        
        Ncf_list.append(Ncf)
        Nuf_list.append(Nuf)
        Ncs_list.append(Ncs)
        d['BuggyVersion']=i+1
        d['PassedTests']=passed
        d['FailedTests']=failed
        d['FailedTestCoverStatement']=Ncf
        d['FailedTestDontCoverStatement']=Nuf
        d['SuccesfulTestCoverStatement']=Ncs
        d['Density']=density
        d['Uniqueness']=uniqueness
        d['Diversity']=diversity
        
        matchList.append(d)
    print(passlist)
    print(faillist)
    print(Ncf_list)
    return matchList 



In [2]:

## The method-data file where the gzoltar files are included should be also in the same directory
# Change path and project name accordingly
path=os.getcwd()
gzoltarpath= os.getcwd() +"/method-data"
projectName="Math"
versionCount=106

matchList=get_suspiciousness_and_rank(gzoltarpath,projectName)


#save to results as csv to current path
currentPath=os.getcwd()
csv_file=currentPath+"/output_"+  projectName + ".csv"

csv_columns=['BuggyVersion','PassedTests','FailedTests','FailedTestCoverStatement','FailedTestDontCoverStatement','SuccesfulTestCoverStatement', 'Density', 'Uniqueness', 'Diversity']

matchList_writer(csv_file,csv_columns,matchList)

[303, 15, 1059, 50, 240, 200, 167, 21, 34, 350, 9, 653, 87, 91, 2739, 2736, 80, 25, 24, 23, 22, 85, 16, 15, 6, 208, 208, 25, 73, 2, 309, 22, 27, 23, 8, 82, 283, 16, 68, 88, 220, 26, 72, 161, 432, 188, 188, 35, 34, 35, 35, 27, 166, 62, 87, 4, 3, 7, 1390, 25, 11, 1, 654, 52, 74, 6, 1, 42, 13, 15, 112, 151, 165, 55, 7, 15, 344, 115, 332, 35, 34, 13, 16, 3, 91, 10, 15, 14, 6, 6, 55, 196, 195, 195, 21, 147, 58, 111, 275, 47, 68, 24, 19, 90, 8, 15]
[2, 1, 1, 2, 1, 28, 1, 1, 1, 1, 1, 3, 1, 1, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 3, 1, 2, 1, 1, 1, 4, 2, 4, 1, 1, 1, 1, 1, 6, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 4, 1, 2, 2, 1, 2, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 6, 1, 1, 1, 1]
[29, 28, 1, 66, 4, 3883, 130, 22, 15, 21, 92, 113, 60, 40, 22, 50, 35, 165, 56, 164, 94, 43, 52, 51, 8, 12, 5, 87, 120, 47, 117, 233, 94, 10, 8, 9, 66, 49, 70, 41, 20, 84, 220, 125, 3, 23, 26, 61, 34, 45, 54, 15, 7, 55, 5, 25, 15, 81